# Exchange HAFNIUM Web Shells

A quick notebook looking at webshells disclosed by Microsoft in their MSTIC feed:

https://raw.githubusercontent.com/Azure/Azure-Sentinel/master/Sample%20Data/Feeds/MSTICIoCs-ExchangeServerVulnerabilitiesDisclosedMarch2021.json

In [ ]:
%load_ext dotenv
%dotenv

import os

API_KEY = "API_KEY_HERE"
VT_LIMIT = 10
TARGET_DIR = "/tmp/samples"
FEED_URL = "https://raw.githubusercontent.com/Azure/Azure-Sentinel/master/Sample%20Data/Feeds/MSTICIoCs-ExchangeServerVulnerabilitiesDisclosedMarch2021.json"

# Loaded via .env
if os.getenv('API_KEY') != None:
    API_KEY = os.getenv('API_KEY')
    print("API_KEY loaded from .env")

Now we can ingest the data provided and parse for sample hashes.

In [ ]:
import pandas as pd
import requests
import json

response = json.loads(requests.get(FEED_URL).text)
feed = pd.json_normalize(response).query("IndicatorType == 'sha256'")
pd.set_option('max_colwidth', 0)
display(feed)

And finally we download the samples where available and view the results:

In [ ]:
import sys
import nest_asyncio
import ipywidgets as widgets
import pandas as pd

nest_asyncio.apply()
sys.path.append('../src/')

from helpers import vthelper, vba

results = []

# Retrieve search results from VT and download each sample to TARGET_DIR
vt = vthelper.VirusTotal(API_KEY, limit=VT_LIMIT)
for sampleHash in feed['Indicator']:
    for result in vt.downloadSearchResults(sampleHash, TARGET_DIR):
        results.append((result.sha256, result.tags))
    
# Display our results
pd.set_option('max_colwidth', 800)
df = pd.DataFrame(data=results, columns=["Hash","Tags"])
display(df)

# results contains 3 elements:
# [0] - SHA256 hash
# [1] - VT Tags

In [ ]:
tabs = []
for result in results:
    tabs.append(widgets.Output())
    with tabs[-1]:
        with open(f"{TARGET_DIR}/{result[0]}") as fd:
            data = fd.read()
            display(data)
            
out = widgets.Tab(tabs)
for i in range(len(results)):
    out.set_title(i, results[i][0])

display(out)

In [ ]:
stored = []
for i in range(len(results)):
    stored.append({"path": f"{TARGET_DIR}/{results[i][0]}", "hash": results[i][0]})

%store stored